<a href="https://colab.research.google.com/github/acoiman/pdt/blob/main/asthma_mortality/notebooks/R/01.Asthma_Mortality_INLABRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🕰️ Predicting Asthma Mortality Rate using R-INLA

The integrated nested Laplace approximation (INLA) is a method for approximate Bayesian inference. R-INLA is a package in R that do approximate Bayesian inference for Latent Gaussian Models. In this notebook  we estimate the Normalized Asthma Mortality Rate in Aggentina, from 2001 to 2022 using the R-INLA package

##📦 Import and install required libraries

In [ ]:
library(SpatialEpi)
library(sf)
library(IRdisplay)
library(leaflet)
library(htmlwidgets)
library(htmltools)
library(spdep)
library(INLA)
library(inlabru)
library(ggplot2)
library(dplyr)
library(tidyr)

In [ ]:
setwd("work/pdt")

In [ ]:
# display current work directory
getwd()

In [ ]:
 # embed function to display tmap output
# Created by cromulent
# https://stackoverflow.com/questions/46770320/error-html-widgets-cannot-be-represented-in-plain-text
# https://colab.research.google.com/github/r-spatial/rgee/blob/examples/rgee_colab.ipynb
ee_embed = function(x, height = 450) {
    tmp = tempfile(fileext = ".html")
    htmlwidgets::saveWidget(x@map, tmp)
    rawHTML = base64enc::dataURI(mime = "text/html;charset=utf-8", file = tmp)
    IRdisplay::display_html(paste("<iframe src=", rawHTML, "width=100% height=", height, "id=","igraph", "scrolling=","no","seamless=","seamless", "frameBorder=","0","></iframe>", sep = "\""))
    unlink(tmp)
}

In [ ]:
# embed function to display leaflet output
leaflet_embed <- function(map, height = 450) {
  tmp <- tempfile(fileext = ".html")

  # Guardar el widget Leaflet en un archivo temporal
  htmlwidgets::saveWidget(map, tmp, selfcontained = TRUE)

  # Codificar como Base64
  rawHTML <- base64enc::dataURI(
    mime = "text/html;charset=utf-8",
    file = tmp
  )

  # Mostrar iframe embebido
  IRdisplay::display_html(
    paste(
      "<iframe src=", rawHTML,
      "width=100% height=", height,
      "id=", "lmap",
      "scrolling=", "no",
      "seamless=", "seamless",
      "frameBorder=", "0",
      "></iframe>",
      sep = "\""
    )
  )

  # Borrar archivo temporal
  unlink(tmp)
}

## 💾 Load and transform the dataset

In [ ]:
# Load data
gdf <- st_read("asthma_mortality/data/gpkg/data.gpkg")

In [ ]:
# check data class
class(gdf)

In [ ]:
# visualize data
head(gdf)

In [ ]:
# Reshape gdf to ts long format,  exclude PDPM25
years <- 2001:2022

records <- lapply(1:nrow(gdf), function(i) {
  row <- gdf[i, ]
  iddpto <- row$IDDPTO
  geometry <- st_geometry(row)
  lapply(years, function(year) {
    data.frame(
      IDDPTO = iddpto,
      YEAR = year,
      A = if (!is.null(row[[paste0("A_", year)]])) row[[paste0("A_", year)]] else NA, # A= population
      C = if (!is.null(row[[paste0("C_", year)]])) row[[paste0("C_", year)]] else NA, # C= number of cases
      CA = if (!is.null(row[[paste0("CA_", year)]])) row[[paste0("CA_", year)]] else NA, # CA = NAMR
      PM25 = if (!is.null(row[[paste0("PM25_", year)]])) row[[paste0("PM25_", year)]] else NA, # PM25 = Particulate matter
      NBA = if (!is.null(row[[paste0("NBA_", year)]])) row[[paste0("NBA_", year)]] else NA, # NBA = Normalized Burned Area
      PD = if (!is.null(row[[paste0("PD_", year)]])) row[[paste0("PD_", year)]] else NA, # PD = Polulation Desity
      #PDPM25 = if (!is.null(row[[paste0("PDPM25_", year)]])) row[[paste0("PDPM25_", year)]] else NA, # PD*PM25
      NAGRT = if (!is.null(row[[paste0("NAGRT_", year)]])) row[[paste0("NAGRT_", year)]] else NA, # NAGRT= LULC changes to  agricultural and livestock class
      NNWVT = if (!is.null(row[[paste0("NNWVT_", year)]])) row[[paste0("NNWVT_", year)]] else NA, # NNWVT= LULC changes from natural wooded vegetation class
      NBUT = if (!is.null(row[[paste0("NBUT_", year)]])) row[[paste0("NBUT_", year)]] else NA, # NBUT= LULC changes to  Built-up class
      ELEV = if (!is.null(row[[paste0("ELEV_", year)]])) row[[paste0("ELEV_", year)]] else NA, # ELEV= elevation
      geometry = geometry
    )
  }) %>% bind_rows()
}) %>% bind_rows()

In [ ]:
# Turn it back into an sf object
sf_ts <- st_as_sf(records)

In [ ]:
# visualize sf
head(sf_ts)

##💻 Computing the expected (E) counts per year

In spatio-temporal settings where disease counts are observed over time, we can use spatio-temporal models that account not only for spatial structure but also for temporal correlations and spatio-temporal interactions [1][2]. Specifically, counts  $Yij$ observed in area $i$ and time $j$ are modeled using the Bernardinelli [3] model as:

\begin{align}
Y_{ij}\sim Po(E_{ij} \theta_{ij}),\ i=1,\ldots,I, \ j=1,\ldots,J,
\end{align}

where $\theta_{ij}$ is the relative risk and $E_{ij}$ s the expected number of cases in area $i$ and time $j$. This means that the observed number of cases follows a Poisson distribution with a mean equal to expected cases times the relative risk.




 The expected counts $Ei$ represent the total number of cases that one would expect if the population of area $i$ behaved the way the standard (or regional) population behaves [4].The expected (E) counts per year is calculated  as:

\begin{align}
E_i= r^{(s)} n^{(i)}
\end{align}


Where:
$r(s)$ is the rate in the standard population-overall rate-(total number of cases (C) divided by total population (A) in all area for a given year), and  $n(i)$ is the population of area i (A) for a given year.

In [ ]:
# calculate expected cases by year
dE <- sf_ts %>%
  group_by(YEAR) %>%
  mutate(
    overall_rate = sum(C, na.rm = TRUE) / sum(A, na.rm = TRUE),
    E = overall_rate * A
  ) %>%
  ungroup()

In [ ]:
# visualize data frame
head(dE)

In [ ]:
# drop overall rate column
dE <- dE %>%
  select(-overall_rate)#%>%
  #select(-A)

In [ ]:
head(dE)

In [ ]:
# rename columns
d <- dE %>%
  rename(county = IDDPTO, Y = C, year=YEAR)

In [ ]:
# visualize df
head(d)

In [ ]:
# copy d as map to preserve the original dataset
map <- d

## 👩‍💻 Modeling

The $\log(\theta_{ij})$ of the Bernardinelli Model is expressed as a sum of several components including spatial and temporal structures that take into account spatial and spatio-temporal correlation:

\begin{align}
log(\theta_{ij})=\alpha + u_i + v_i + (\beta + \delta_i) \times t_j.
\end{align}

Here, $\alpha$ denotes the intercept, $u_i+v_i$ is an area random effect, $\beta$ is an area random effect,  and  $\delta_i$ is an interaction between space and time representing the difference between the global trend $\beta$ and the area specific trend. Generally, we model $u_i$ (spatial random effects) and $\delta_i$ with CA (Conditional AutoRegressive) distribution, and $v_i
$ (unstructured random effects) as independent and identically distributed (i.i.d) normal variables. This model allows each of the areas to have its own intercept $\alpha+u_i+v_i$ and its own linear trend given by  $\beta + \delta_i $ [4].

Conditional AutoRegressive (CAR) distributions are a fundamental tool in spatial statistics, particularly useful for modeling spatial dependencies in areal data. These models are characterized by their ability to capture spatial correlation through a graph-based structure, where the conditional distribution of a variable depends only on its neighbors. This makes CAR models particularly suitable for Bayesian hierarchical models, where they serve as prior distributions for spatial random effects [5][6].

The Independent and Identically Distributed (i.i.d) normal variables are random variables that all follow the same Normal (Gaussian) distribution, meaning they have the same mean, same variance and independent [7].



### Neighborhood matrix

First, we will create a neighborhood matrix needed to define the spatial random effect

In [ ]:
# create a neighborhood matrix needed to define the spatial random
#nb <- poly2nb(d)

In [ ]:
# visualize neighborhood matrix
#head(nb)

In [ ]:
# Output spatial neighbours for INLA
#nb2INLA("asthma_mortality/data/adj/map.adj", nb)

In [ ]:
# read the file created using the inla.read.graph() function
g <- inla.read.graph(filename = "asthma_mortality/data/adj/map.adj")

### Inference using INLABRU

The INLABRU R package implements innovative methods to model spatial distribution and change from ecological survey data.  It develops the Integrated Nested Laplace Approximation (INLA) methods for fitting complex spatial models to data obtained from surveys on which the probability of detecting population members is unknown. [8]

We will split our data for training (d_train) and testing and predictions (d_test) datasets. Training will be from 2001-2021, and testing for 2022

In [ ]:
# select data for training from 2001 to 2021
d_train <- d %>%
  filter(year >= 2001, year <= 2021)

In [ ]:
# select data for testing and prediction 2022
d_test <- d %>%
  filter(year == 2022)

Next, we create the index vectors for the counties and years that will be used to specify the random effects of the model:

* idarea is the vector with the indices of counties and has elements 1 to 511 (number of deparments).

* idtime is the vector with the indices of years and has elements 1 to 20 (number of years).

We also create a second index vector for counties (idarea1) by replicating idarea. We do this because we need to use the index vector of the areas in two different random effects, and in R-INLA variables can be associated with an f() function only [4].

In [ ]:
# we create the index vectors for the counties and years
d_train$idarea <- as.numeric(as.factor(d_train$county))
d_train$idarea1 <- d_train$idarea
d_train$idtime <- 1 + d_train$year - min(d_train$year)

In [ ]:
# visualize data
head(d_train)

#### Zero-inflated model  type 0

In this section we will explore zero-inflated type 0 model for likelihood

The Zero-inflated model type 0  formula is:

\begin{align}
Prob(y | . . .) = p × 1_{[y=0]} + (1 − p) × Poisson(y | y > 0)
\end{align}

where:
* y= observations
* p = probability

The Type 0 zero-inflated Poisson model says that observed zeros may come either from:

* a structural-zero process that returns a zero automatically, or
* the usual Poisson process tahat returns zero counts or positive counts

ZIP Type 0 = structural zeros with probability p + usual Poisson distribution with probability (1–p). Positive counts can only come from the Poisson part; zeros can come from both parts.

##### Besag-York-Mollié (BYM) for spatial effects, and Independent and Identically Distributed (i.i.d)  for space-time random effects

Let's explain the implentation of these models in the formula of the Bernardinelli model:

Formula of the Bernardinelli model:

* Training Features: PM25+NBA+PD+NAGRT+NNWVT+NBUT+ELEV

* $Y$ is our responce variables, number of asthma death

* The intercept $\alpha$ is included by default
* se *(idarea, model = "bym", graph = g)* corresponds to $u_i+v_i$.
  * $u_i$ is the structured spatial effect (captures correlation among neighboring areas).
  * $v_I$ is the unstructured spatial effect (area-specific noise, independent).
* ste *(idarea1, idtime, model = "iid")* is the interaction term $\delta_i \times t_j$ (Gaussian random effect).
  * This is an unstructured space-time interaction (each area-time pair is independent). If we want structured space-time interaction we need to use models like Random Walk of order 1 or 2 (RW1, RW2), AR(1) temporal structure.
  * It is the area-specific deviation from the global trend (so each area can have its own slope in time).
  * $\delta_i$ is an interaction between space and time representing the difference between the global trend $\beta$ and the area specific trend

* idtime is the he global linear time trend $\delta_i \times t_j$ [4].


In [ ]:
# formula of the Bernardinelli model zeroinflatedpoisson0
formula_zip00 <- Y ~ PM25+NBA+PD+NAGRT+NNWVT+NBUT+ELEV+
  se(idarea, model = "bym", graph = g)+
  ste(idarea1, idtime, model = "iid")+
  idtime

 Now, we call bru()-a wrapper for INLA::inla- specifying the formula, the family model, the data, and the expected cases.

In [ ]:
# call bru() to fit model
result_zip00 <- bru(formula_zip00,
  family = "zeroinflatedpoisson0", data = d_train, E = d_train$E,
  options = list(control.compute = list(dic = TRUE), verbose = TRUE, num.threads = "6:2")
)

Let's explain what is the mean of the formula and fitting method $bru()$ in the context of Bayes Theorem:



\begin{align}
p(Hip\, |\, Ev) = \frac{p(Ev\,|\,Hip) \quad p(Hip)}{p(Ev\,|\,Hip)+p(Hip) + p(Ev\,|\,Hip')+p(Hip') }
\end{align}

Where:

$p(Ev\,|\,Hip)$ is the **likelihood**, the probability of the data (Ev) given the parameters (Hip).

$p(Hip)$ is the **prior** distribution on the unknown parameters.

$p(Ev\,|\,Hip)+p(Hip) + p(Ev\,|\,Hip')+p(Hip')$ is the **Evidence/Marginal likelihood**

<br>

In our formula:

**Likelihood:**

Likelihood is family = "zeroinflatedpoisson0" + data +  E

We say to INLA:
* the response $Y$ follows a Type 0 zero-inflated Poisson model
* with the expected counts $E$ multiplied by the relative risk

Where the “data”  $Ev$ in Bayes theorem corresponds to our observed Y.

**Prior**

Prior is se(...), ste(...), and the default priors on fixed effects

In inlabru, the priors come from the model components:

* spatial structured + unstructured (“bym” model):
  * $se(idarea, model = "bym", graph = g)$
* BYM = Besag–York–Mollié model, contains:
  * A structured spatial effect (ICAR-Intrinsic Conditional Autoregressive model-prior). ICAR is the standard prior used for structured spatial random effects in disease mapping, including the BYM model.
  * An unstructured random effect (iid Gaussian prior)
  * The prior of the hiperparameter ${\theta}$ (the precision parameter of the structured spatial effect) is loggamma with shape = 1 and rate = 0.0005
  * These are prior distributions over the spatial field.
* Space–time interaction (iid)
  * $ste(idarea1, idtime, model = "iid")$: This is a prior for the unstructured space–time random effect (normal iid)
Fixed effects priors (automatic): inlabru/INLA automatically assigns Gaussian priors (typically mean 0, large variance) to regression coefficients (features)

All of these together define $p(Hip)$, the prior.

**Evidence / Marginal Likelihood (INLA internal normalization)**

In Bayes theorem:

\begin{align}
{p(Ev\,|\,Hip)+p(Hip) + p(Ev\,|\,Hip')+p(Hip') }
\end{align}

It is computed internally by INLA using Laplace approximations, and appears as:

* the log marginal likelihood (log marginal likelihood)
* internal integrals over hyperparameters
* We do not write code for this — INLA computes it automatically

**Posterior**

This is what INLA computes, for example $result-zip0$ object, it contains:

* posterior means

* posterior standard deviations

* posterior marginals for:

  * fixed effects

  * random effects (BYM, iid, space–time)

  * hyperparameters (range, precision, phi in zipo)

* fitted values

* predictions


##### Besag-York-Mollié 2 (BYM2), Penalized Complexity (PC) priors  for spatial effects, and Independent and Identically Distributed (i.i.d)  for space-time random effects

We will fit a Zero-inflated model type 0 model with Bym2 model as a prior. We will change the default prior (hiperparameter) for the precision of the random effects $u_i$ $1/\sigma^2 \sim Gamma(1, 5 \times  10^{-5})$ a Penalized Complexity (PC) prior on the standard deviation $\sigma$. We will specify that the probability of $\sigma$ being greater than 0.5 is  equal to 0.01: $P(\sigma > 1) = 0.01$ (99% probability that the marginal SD < 0.5). In that way we allow moderate spatial variation, and avoid overfitting.

In [ ]:
# set pc.prec value
prior.prec <- list(prec = list(prior = "pc.prec",
                               param = c(0.5, 0.01)))

In [ ]:
# formula bym2
formula_zip01 <- Y ~ PM25+NBA+PD+NAGRT+NNWVT+NBUT+ELEV+
  se(idarea, model = "bym2", graph = g, hyper = prior.prec)+
  ste(idarea1, idtime, model = "iid")+
  idtime

In [ ]:
# call bru() to fit model with bym2
result_zip01 <- bru(formula_zip01,
  family = "zeroinflatedpoisson0", data = d_train, E = d_train$E,
  options = list(control.compute = list(dic = TRUE), verbose = TRUE, num.threads = "6:2")
)

##### Besag-York-Mollié 2 (BYM2), Penalized Complexity (PC) priors  for spatial effects, and Independent and  Random Walk of order 2 (RW2)  for space-time random effects


Next, we will fit a Zero-inflated model type 0 model with Bym2 model as a prior, but this time we specify that the temporal evolution follows a Random Walk of order 2 (RW2). RW2 does not force linear growth; instead, it allows:

* smooth accelerations,

* smooth decelerations,

* long-term smooth changes.


In [ ]:
# formula bym2
formula_zip02 <- Y ~ PM25+NBA+PD+NAGRT+NNWVT+NBUT+ELEV+
  se(idarea, model = "bym2", graph = g, hyper = prior.prec)+
  ste(idarea1, idtime, model = "rw2")+
  idtime

In [ ]:
# call bru() to fit model with bym2
result_zip02 <- bru(formula_zip02,
  family = "zeroinflatedpoisson0", data = d_train, E = d_train$E,
  options = list(control.compute = list(dic = TRUE), verbose = TRUE, num.threads = "6:2")
)

#### Zero-inflated model  type 1

In this section we will explore zero-inflated type 0 model for likelihood

The Zero-inflated model type 0  formula is:

\begin{align}
Prob(y | . . .) = p × 1_{[y=0]} + (1 − p) × Poisson(y)
\end{align}

where:
* y= observations
* p = probability

The Type 1 zero-inflated Poisson model says that observed zeros may come from:

* the usual Poisson process that returns zero counts or positive counts

ZIP Type 1 does not add extra zeros; only shrinks positive counts, leaving zeros purely from Poisson.

##### Besag-York-Mollié (BYM) for spatial effects, and Independent and Identically Distributed (i.i.d)  for space-time random effects

In [ ]:
# formula of the Bernardinelli model zeroinflatedpoisson0
formula_zip11 <- Y ~ PM25+NBA+PD+NAGRT+NNWVT+NBUT+ELEV+
  se(idarea, model = "bym", graph = g)+
  ste(idarea1, idtime, model = "iid")+
  idtime

In [ ]:
# call bru() to fit model
result_zip11 <- bru(formula_zip11,
  family = "zeroinflatedpoisson1", data = d_train, E = d_train$E,
  options = list(control.compute = list(dic = TRUE), verbose = TRUE, num.threads = "6:2")
)

##### Besag-York-Mollié 2 (BYM2), Penalized Complexity (PC) priors  for spatial effects, and Independent and Identically Distributed (i.i.d)  for space-time random effects

In [ ]:
# set pc.prec value
prior.prec <- list(prec = list(prior = "pc.prec",
                               param = c(0.5, 0.01)))

In [ ]:
# formula bym2
formula_zip12 <- Y ~ PM25+NBA+PD+NAGRT+NNWVT+NBUT+ELEV+
  se(idarea, model = "bym2", graph = g, hyper = prior.prec)+
  ste(idarea1, idtime, model = "iid")+
  idtime

In [ ]:
# call bru() to fit model with bym2
result_zip12 <- bru(formula_zip12,
  family = "zeroinflatedpoisson1", data = d_train, E = d_train$E,
  options = list(control.compute = list(dic = TRUE), verbose = TRUE,num.threads = "6:2")
)

##### Besag-York-Mollié 2 (BYM2), Penalized Complexity (PC) priors  for spatial effects, and Independent and  Random Walk of order 2 (RW2)  for space-time random effects


In [ ]:
# formula bym2
formula_zip13 <- Y ~ PM25+NBA+PD+NAGRT+NNWVT+NBUT+ELEV+
  se(idarea, model = "bym2", graph = g, hyper = prior.prec)+
  ste(idarea1, idtime, model = "rw2")+
  idtime

In [ ]:
# call bru() to fit model with bym2
result_zip13 <- bru(formula_zip13,
  family = "zeroinflatedpoisson1", data = d_train, E = d_train$E,
  options = list(control.compute = list(dic = TRUE), verbose = TRUE, num.threads = "6:2")
)

#### Picking the best model

In [ ]:
deltaIC(result_zip00, result_zip01, result_zip02, result_zip11, result_zip12, result_zip13)

Based on the The deviance information criterion (DIC) the best model withthe lower DIC is the model using  "zeroinflatedpoisson1" as a likelihood, Besag-York-Mollié 2 (BYM2), Penalized Complexity (PC) priors  for spatial effects, and Independent and  Random Walk of order 2 (RW2)  for space-time random effects

In [ ]:
# model summary
summary(result_zip13)

* The zero-inflation parameter is extremely small (~0.005), indicating almost no excess zeros beyond what the Poisson process already predicts—consistent with ZIP-1, where zeros arise from the Poisson distribution.

* Fixed effects for environmental, demographic, and land-use variables are very small, with all 95% credible intervals overlapping zero. NBUT shows a weak positive association with mortality (0.003, CI: 0.000–0.005), and NNWVT shows a weak negative association (–0.002, CI: –0.005–0.000).

* The spatial BYM2 component shows relatively low residual spatial variability, reflected by a moderate precision (~6.4). The phi value (φ ≈ 0.74) indicates that most of the remaining spatial heterogeneity is structured, meaning neighboring areas share similar mortality patterns.

* The very high precision of the RW2 temporal interaction (~188,000) indicates that the temporal effect is nearly flat, with little unexplained temporal variation after accounting for other components. This means the data contain minimal temporal structure.



### Predicting Asthma Mortality Rate in 2022

The key idea is that in epidemiology and disease mapping, relative risk (RR) is by definition the ratio:

\begin{align}
RR= \frac{Expected(or \ predicted) cases}  {Expected \ cases}
\end{align}

So, to get predicted counts we will use:  

\begin{align}
Predicted\ Counts = Expected\ cases × Relative\ Risk
\end{align}

In [ ]:
# copy d_test
d_test13 <- d_test

In [ ]:
# we create the index vectors for the counties and years
d_test13$idarea <- as.numeric(as.factor(d_test13$county))
d_test13$idarea1 <- d_test13$idarea
d_test13$idtime <- 1 + d_test13$year - min(d_test13$year)

In [ ]:
# visualize dataframe
tail(d_test13)

In [ ]:
pred_zip13 <- predict(
  result_zip13,
  newdata = d_test13,
  formula = ~ exp(Intercept +
                  PM25 + NBA + PD + NAGRT + NNWVT + NBUT + ELEV +
                  se +
                  ste +
                  idtime)
)


In [ ]:
# visualize dataframe
head(pred_zip13)

In [ ]:
# calculate predicted counts, multiply the predicted RR by the true Expected counts
d_test13$Y_pred <- pred_zip13$mean * d_test13$E

In [ ]:
# visualize dataframe
head(d_test13)

In [ ]:
# calculate de CA pred (predicted NAMR)
d_test13$CA_pred <- round((d_test13$Y_pred / d_test13$A) * 100000, 2)

In [ ]:
# vislialize dataframe
head(d_test13)

In [ ]:
# MAE
MAE <- mean(abs(d_test13$CA - d_test13$CA_pred))

# RMSE
RMSE <- sqrt(mean((d_test13$CA - d_test13$CA_pred)^2))

MAE
RMSE


In [ ]:
# calculate R2 score
caret::R2(d_test13$CA_pred, d_test13$CA)

In [ ]:
# plot predicted vs true values
plot(d_test13$CA, d_test13$CA_pred,
     xlab = "Real Values (CA)",
     ylab = "Predicted Values (CA_pred)",
     main = "Predicted vs Real Values",
     pch = 19)

abline(a = 0, b = 1, col = "red", lwd = 2)  # 1:1 reference lin

## 📚 References

1. Martínez-Beneito, Miguel A., Antonio López-Quílez, and Paloma Botella-Rocamora. 2008. “An Autoregressive Approach to Spatio‐temporal Disease Mapping.” Statistics and Medicine 27: 2874–89.

2. Ugarte, María Dolores, Aritz Adin, Tomas Goicoa, and Ana Fernandez Militino. 2014. “On fitting spatio-temporal disease mapping models using approximate Bayesian inference.” Statistical Methods in Medical Research 23

3. Bernardinelli, L., Clayton, D., Pascutto, C., Montomoli, C., Ghislandi, M., & Songini, M. (1995). Bayesian analysis of space—time variation in disease risk. Statistics in medicine, 14(21‐22), 2433-2443.

4. Moraga, P. (2019). Geospatial health data: Modeling and visualization with R-INLA and shiny. Chapman and Hall/CRC. (6): 507–30.

5.  Lee, D., &#38; Mitchell, R. (2012). Locally adaptive spatial smoothing using conditional autoregressive models. <i>arXiv: Applications</i>. http://export.arxiv.org/pdf/1205.3641

6.   Sun, D., Tsutakawa, R. K., &#38; Speckman, P. L. (1999). Posterior distribution of hierarchical models using CAR(1) distributions. <i>Biometrika</i>, <i>86</i>(2), 341–350. https://doi.org/10.1093/BIOMET/86.2.341

7. Benhamou, E., Guez, B., &#38; Paris, N. (2018). Three remarkable properties of the Normal distribution. <i>arXiv: Probability</i>. https://arxiv.org/abs/1810.01768

[8] Bachl, F. E. (2025). inlabru. Inlabru. https://sites.google.com/inlabru.org/inlabru






## 💊 Supplementary material

#### Precision and variance relationship

\begin{align}
precision = {\tau}= \frac1{\sigma^2}
\end{align}


* High precision → small variance → effect is almost flat (shrunk heavily toward 0/mean)

* Low precision → large variance → effect is wiggly or strong

**Low precision**

$ {\tau} < 1$

* Large variance

* Strong spatial/temporal variability

* Rough, wiggly random effect

* Model finds a real effect in data

**Medium variability**

$1 < {\tau} < 10$

* Medium variability

* Random effects present but not strong

* Partially smoothed field

**High precision**

$1 < {\tau} > 10$

* Very small variance

* Random effect collapses toward zero or a straight line

* Model finds almost no residual structure


**Very high precision**

$1 < {\tau} > 1000 or 10000$

This means:

* The effect is essentially flat or constant

* The random field contributes almost nothing to the model

* The model shrinks the random effect to zero

